In [1]:
import tensorflow as tf
import numpy as np
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

Vi starter med at loaded den givne data ind. Her er dataen sepereret i lyd med musik og lyd uden musik. Det skal vi så sætte sammen i et array og lave en array som indeholder om hvilken type hver styk data er. Her kan vi bruge numpy siden hver fil er i .npy format

In [2]:
music_data = np.load("music_data.npy")
other_data = np.load("other_data.npy")
unknown_data = np.load("test_data.npy")

Data layoutet er her $10500$ 2d arrays af $30\times79$ som repræsentere et log-mel spektogram udregnet fra den givne lyd sample. $x$ dimensionen er frekvens båndene mens $y$ er tids rammen

Nu skal vi sætte `music_data` og `other_data` sammen til et array og lave en array som indeholder hvilken type hver lyd sample er. Vi vil også gerne have noget test data så vi kan evaluere hvor godt netværket er uden at overfitte Det gør vi ved at efterlade 1000 fra `music_dat` og `other_data`

In [3]:
import random
music_train_data = music_data[0: -1000]
other_train_data = other_data[0: -1000]
train_data = (
	np.concatenate((music_train_data, other_train_data)),
	np.concatenate((
		np.full((len(music_train_data), 2), (1, 0)), 
		np.full((len(other_train_data), 2), (0, 1))
	))
)
train_data = unison_shuffled_copies(train_data[0], train_data[1])

music_test_data = music_data[len(music_data) - 1000:None]
other_test_data = other_data[len(other_data) - 1000:None]
test_data = (
	np.concatenate((music_test_data, other_test_data)),
	np.concatenate((
		np.full((len(music_test_data), 2), (1, 0)),
		np.full((len(other_test_data), 2), (0, 1))
	))
)

test_data = unison_shuffled_copies(test_data[0], test_data[1])

We need to shuffle the data before training

In [4]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Reshape((30, 79, 1), input_shape = (30, 79)),
	tf.keras.layers.Conv2D(3, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(pool_size = (1,2)),
	tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Flatten(),
	#tf.keras.layers.Dense(128, activation = "relu"),
	#tf.keras.layers.Dropout(.5),
	tf.keras.layers.Dense(2, activation = "softmax")
])

model.build(input_shape=(30, 79))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 30, 79, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 77, 3)         30        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 38, 3)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 36, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 16, 64)        1

2022-04-16 15:34:43.530920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 15:34:43.561540: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-04-16 15:34:43.561568: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-04-16 15:34:43.562113: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Nu har vi lavet dataset til test og træning, nu skal vi sætte vores model op

Nu hvor vi har en model, så skal den bare trænes

In [5]:
predictions = model(train_data[0][:10]).numpy()
predictions

array([[0.48132142, 0.5186786 ],
       [0.5068126 , 0.4931874 ],
       [0.39786944, 0.60213053],
       [0.47222584, 0.52777416],
       [0.577847  , 0.4221531 ],
       [0.5255568 , 0.47444323],
       [0.3095315 , 0.6904685 ],
       [0.5695462 , 0.4304538 ],
       [0.66269857, 0.33730143],
       [0.52572453, 0.4742755 ]], dtype=float32)

Nu laver vi en loss funktion, siden vi kun har et output så bruger vi binarycrossentropy

In [6]:
model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

Så træner vi modellen med dataen over 10 epochs

In [7]:
history = model.fit(train_data[0], train_data[1], epochs=10)

Epoch 1/10
 33/594 [>.............................] - ETA: 9s - loss: 0.8630 - accuracy: 0.5805

KeyboardInterrupt: 

In [ ]:
train_evaluation = model.evaluate(train_data[0], train_data[1])
test_evaluation = model.evaluate(test_data[0], test_data[1])
print("Training set accuracy %.3f%% and loss: %.3f%%" % 
	(train_evaluation[1] * 100, train_evaluation[0] * 100))
print("Test set accuracy %.3f%% and loss: %.3f%%" % 
	(test_evaluation[1] * 100, test_evaluation[0] * 100))

In [ ]:
data = model(unknown_data).numpy()

In [ ]:
f = open("predictions.txt", "w")

first = True

for prediction in data:
	if first:
		first = False
	else:
		f.write("\n")
	if prediction[0] > prediction[1]:
		f.write("1")
	else:
		f.write("0")
f.close()

For lige at være sikker på modellen giver det vi forventer, så printer vi det lige her

In [ ]:
pred = model(music_data[:1]).numpy()[0]
print("%.3f %.3f" % (pred[0], pred[1]))
pred = model(other_data[:1]).numpy()[0]
print("%.3f %.3f" % (pred[0], pred[1]))